In [ ]:
!nvidia-smi

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import torch
import h5py
import onnxruntime as ort
import openvino as ov
import re

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
from copy import deepcopy
from pprint import pprint

# from code_base.utils import parallel_librosa_load, groupby_np_array, stack_and_max_by_samples, macro_f1_similarity, N_CLASSES_2021_2022, N_CLASSES_2021, comp_metric, N_CLASSES_XC_LIGIT_SHORTEN, N_CLASSES_XC_LIGIT_EVEN_SHORTEN
# from code_base.utils.constants import SAMPLE_RATE
from code_base.utils.onnx_utils import ONNXEnsemble, convert_to_onnx
from code_base.models import WaveCNNClasifier, WaveCNNAttenClasifier
from code_base.datasets import WaveDataset, WaveAllFileDataset
from code_base.utils.swa import avarage_weights, delete_prefix_from_chkp
from code_base.inefernce import BirdsInference
from code_base.utils import load_json, compose_submission_dataframe, groupby_np_array, stack_and_max_by_samples, write_json
from code_base.utils.metrics import score_numpy
%matplotlib inline


/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


`speechbrain` was not imported
`LEAF` was not imported


/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


# Export Models

In [ ]:
!ls -lt ../logdirs/ | head -20

In [2]:
bird2id_source = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp.json")
bird2id_target = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json")

id2bird_source = {v:k for k,v in bird2id_source.items()}
id2bird_target = {v:k for k,v in bird2id_target.items()}

REARRANGE_INDICES = np.array([
    bird2id_source[id2bird_target[i]] for i in range(len(id2bird_target))
]).astype(int)

MODEL_CLASS = WaveCNNAttenClasifier
TRAIN_PERIOD = 5
STRICT_LOAD = False

def prune_checkpoint_rule(inp_chkp):
    inp_chkp["head.attention.weight"] = inp_chkp["head.attention.weight"][REARRANGE_INDICES]
    inp_chkp["head.attention.bias"] = inp_chkp["head.attention.bias"][REARRANGE_INDICES]
    
    inp_chkp["head.fix_scale.weight"] = inp_chkp["head.fix_scale.weight"][REARRANGE_INDICES]
    inp_chkp["head.fix_scale.bias"] = inp_chkp["head.fix_scale.bias"][REARRANGE_INDICES]

    return inp_chkp

In [3]:
len(bird2id_source)

998

In [4]:
# EXP_NAME = "convnextv2_tiny_fcmae_ft_in22k_in1k_384_And_tf_efficientnetv2_b2_in1k_Pred075_Timewise025"
EXP_NAME = "tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1"
TRAIN_PERIOD = 5
print("Possible checkpoints:\n\n{}".format("\n".join(set([os.path.basename(el) for el in glob(f"../logdirs/{EXP_NAME}/checkpoints/*.ckpt") if "train" not in os.path.basename(el)]))))

Possible checkpoints:




In [ ]:
conf_path = glob(f"../logdirs/{EXP_NAME}/code/*train_configs*.py")
assert len(conf_path) == 1
conf_path = conf_path[0]
!cat {conf_path}

In [5]:
MODELS = [
    {
        "model_config": dict(
            # backbone="convnextv2_tiny.fcmae_ft_in22k_in1k_384",
            # backbone="tf_efficientnetv2_b2.in1k",
            # backbone="tf_efficientnetv2_b0.in1k",
            backbone="tf_efficientnetv2_b1.in1k",
            # backbone="eca_nfnet_l0",
            # add_backbone_config={"drop_path_rate": 0.2},
            mel_spec_paramms={
                "sample_rate": 32000,
                "n_mels": 128,
                "f_min": 20,
                "n_fft": 2048,
                "hop_length": 512,
                "normalized": True,
            },
            head_config={
                "p": 0.5,
                "num_class": 188,
                "train_period": TRAIN_PERIOD,
                "infer_period": TRAIN_PERIOD,
                "output_type": "clipwise_pred_long",
            },
            exportable=True,
        ),
        "exp_name": EXP_NAME,
        #"convnextv2_tiny_fcmae_ft_in22k_in1k_384_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_StrongBackGroundSoundScapeP075_StrongPinkOrGaus075_mixupP075Alpha1_RandomFiltering_balancedSampler_Radamlr1e4_CosBatchLR1e6_Epoch30_FocalLoss_Full_NoDuplsV1",
        "fold": None,
        "chkp_name":"last.ckpt",
        "swa_checkpoint_regex": None, #r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
        "use_sigmoid": True,
        "swa_sort_rule": lambda x: -float(x["valid_roc_auc"]),
        "delete_prefix": "model.",
        "n_swa_models": 1,
        "model_output_key": None,
        # "prune_checkpoint_func": prune_checkpoint_rule
    },
    # {
    #     "model_config": dict(
    #         backbone="maxvit_rmlp_pico_rw_256.sw_in1k",
    #         mel_spec_paramms={
    #             "sample_rate": 32000,
    #             "n_mels": 128,
    #             "f_min": 20,
    #             "n_fft": 2048,
    #             "hop_length": 512,
    #             "normalized": True,
    #         },
    #         spec_resize=(256,256),
    #         spec_augment_config={
    #             "freq_mask": {
    #                 "mask_max_length": 10,
    #                 "mask_max_masks": 3,
    #                 "p": 0.3,
    #                 "inplace": True,
    #             },
    #             "time_mask": {
    #                 "mask_max_length": 20,
    #                 "mask_max_masks": 3,
    #                 "p": 0.3,
    #                 "inplace": True,
    #             },
    #         },
    #         atten_smoothing_config={
    #             "dropout": 0.1,
    #             "num_layers": 1,
    #             "n_steps": 64,
    #         },
    #         head_type="AttHeadSimplified",
    #         head_config={
    #             "p": 0.5,
    #             "num_class": 188,
    #             "omit_pooling": True,
    #             "output_type": "clipwise_pred_long",
    #         },
    #         exportable=True,
    #     ),
    #     "exp_name": "maxvit_rmlp_pico_rw_256_sw_in1k_Exp_FullAtten_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_BackGroundSoundScapeP05_mixupP05_RandomFiltering_balancedSampler_Radamlr3e4_CosBatchLR1e6_Epoch30_SpecAugV1_FocalLoss_Full_NoDuplsV1",
    #     "fold": None,
    #     "chkp_name":"last.ckpt",
    #     "swa_checkpoint_regex": None, #r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    #     "use_sigmoid": True,
    #     "swa_sort_rule": lambda x: -float(x["valid_tresh01_dice3d"]),
    #     "delete_prefix": "model.",
    #     "n_swa_models": 3,
    #     "model_output_key": None,
    #     # "prune_checkpoint_func": prune_checkpoint_rule
    # },
    #  {
    #     "model_config": dict(
    #         backbone="tf_efficientnetv2_b2.in1k",
    #         mel_spec_paramms={
    #             "sample_rate": 32000,
    #             "n_mels": 128,
    #             "f_min": 20,
    #             "n_fft": 2048,
    #             "hop_length": 512,
    #             "normalized": True,
    #         },
    #         head_config={
    #             "p": 0.5,
    #             "num_class": 188,
    #             "train_period": TRAIN_PERIOD,
    #             "infer_period": TRAIN_PERIOD,
    #             "output_type": "clipwise_timewisemax_pred_short",
    #             "infer_framewise_max_coef": 0.25
    #         },
    #         exportable=True,
    #     ),
    #     "exp_name": "tf_efficientnetv2_b2_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_BackGroundSoundScapeP05_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_Full_NoDuplsV1",
    #     "fold": None,
    #     "chkp_name":"last.ckpt",
    #     "swa_checkpoint_regex": None, #r'(?P<key>\w+)=(?P<value>[\d.]+)(?=\.ckpt|$)',
    #     "use_sigmoid": True,
    #     "swa_sort_rule": lambda x: -float(x["valid_tresh01_dice3d"]),
    #     "delete_prefix": "model.",
    #     "n_swa_models": 3,
    #     "model_output_key": None,
    #     # "prune_checkpoint_func": prune_checkpoint_rule
    # },
]

In [6]:
def create_model_and_upload_chkp(
    model_class,
    model_config,
    model_device,
    model_chkp_root,
    model_chkp_basename=None,
    model_chkp_regex=None,
    delete_prefix=None,
    swa_sort_rule=None,
    n_swa_to_take=3,
    prune_checkpoint_func=None
):
    if model_chkp_basename is None:
        basenames = os.listdir(model_chkp_root)
        checkpoints = []
        for el in basenames:
            matches = re.findall(model_chkp_regex, el)
            if not matches:
                continue
            parsed_dict = {key: value for key, value in matches}
            parsed_dict["name"] = el
            checkpoints.append(parsed_dict)
        print("SWA checkpoints")
        pprint(checkpoints)
        checkpoints = sorted(checkpoints, key=swa_sort_rule)
        checkpoints = checkpoints[:n_swa_to_take]
        print("SWA sorted checkpoints")
        pprint(checkpoints)
        if len(checkpoints) > 1:
            checkpoints = [
                torch.load(os.path.join(model_chkp_root, el["name"]), map_location="cpu")["state_dict"] for el in checkpoints
            ]
            t_chkp = avarage_weights(
                nn_weights=checkpoints,
                delete_prefix=delete_prefix
            )
        else:
            chkp_path = os.path.join(model_chkp_root, checkpoints[0]["name"])
            print("vanilla model")
            print("Loading", chkp_path)
            t_chkp = torch.load(
                chkp_path, 
                map_location="cpu"
            )["state_dict"]
            if delete_prefix is not None:
                t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)
    else:
        chkp_path = os.path.join(model_chkp_root, model_chkp_basename)
        print("vanilla model")
        print("Loading", chkp_path)
        t_chkp = torch.load(
            chkp_path, 
            map_location="cpu"
        )["state_dict"]
        if delete_prefix is not None:
            t_chkp = delete_prefix_from_chkp(t_chkp, delete_prefix)

    if prune_checkpoint_func is not None:
        t_chkp = prune_checkpoint_func(t_chkp)
    t_model = model_class(**model_config, device=model_device) 
    print("Missing keys: ", set(t_model.state_dict().keys()) - set(t_chkp))
    print("Extra keys: ",  set(t_chkp) - set(t_model.state_dict().keys()))
    t_model.load_state_dict(t_chkp, strict=False)
    t_model.eval()
    return t_model

In [7]:
# model = [create_model_and_upload_chkp(
#     model_class=MODEL_CLASS,
#     model_config=model_config['model_config'],
#     model_device="cpu",
#     model_chkp_root=f"../logdirs/{model_config['exp_name']}/checkpoints",
#     model_chkp_basename=model_config["chkp_name"] if model_config["swa_checkpoint_regex"] is None else None,
#     model_chkp_regex=model_config.get("swa_checkpoint_regex"),
#     swa_sort_rule=model_config.get("swa_sort_rule"),
#     n_swa_to_take=model_config.get("n_swa_models", 3),
#     delete_prefix=model_config.get("delete_prefix"),
#     prune_checkpoint_func=model_config.get("prune_checkpoint_func")
# ) for model_config in MODELS]

In [8]:

model = [create_model_and_upload_chkp(
    model_class=MODEL_CLASS,
    model_config=MODELS[0]['model_config'],
    model_device="cpu",
    model_chkp_root=f"../logdirs/{MODELS[0]['exp_name']}/fold_{m_i}/checkpoints",
    # model_chkp_root=f"../logdirs/{CONFIG['exp_name']}/checkpoints",
    model_chkp_basename=MODELS[0]["chkp_name"] if MODELS[0]["swa_checkpoint_regex"] is None else None,
    model_chkp_regex=MODELS[0].get("swa_checkpoint_regex"),
    swa_sort_rule=MODELS[0].get("swa_sort_rule"),
    n_swa_to_take=MODELS[0].get("n_swa_models", 3),
    delete_prefix=MODELS[0].get("delete_prefix"),
    prune_checkpoint_func=MODELS[0].get("prune_checkpoint_func")
) for m_i in range(5)]


vanilla model
Loading ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/fold_0/checkpoints/last.ckpt
STFT kernels created, time used = 0.3047 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
vanilla model
Loading ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/fold_1/checkpoints/last.ckpt
STFT kernels created, time used = 0.1195 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
vanilla model
Loading ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/fold_2/checkpoints/last.ckpt
STFT kernels created, time used = 0.0947 seconds


...

Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
vanilla model
Loading ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/fold_3/checkpoints/last.ckpt
STFT kernels created, time used = 0.1196 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}
vanilla model
Loading ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/fold_4/checkpoints/last.ckpt
STFT kernels created, time used = 0.1376 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


Missing keys:  set()
Extra keys:  {'_forward.batch_aug.hann_window'}


In [9]:
# exportable_ensem = ONNXEnsemble(
#     model_class=MODEL_CLASS,
#     configs=[deepcopy(config['model_config']) for config in MODELS],
#     # final_activation="softmax"
#     # avarage_type="gaus"
#     # weights=[0.75,0.4,0.5]
# )
exportable_ensem = ONNXEnsemble(
    model_class=MODEL_CLASS,
    configs=[deepcopy(MODELS[0]['model_config']) for _ in range(5)],
    # final_activation="softmax"
    # avarage_type="gaus"
    # weights=[0.75,0.4,0.5]
)
for model_id in range(len(exportable_ensem.models)):
    exportable_ensem.models[model_id].load_state_dict(model[model_id].state_dict())
exportable_ensem.eval()
convert_to_onnx(
    model_to_convert=exportable_ensem,
    sample_input=torch.randn(5, TRAIN_PERIOD * 32_000),
    base_path=f"../logdirs/{EXP_NAME}/onnx_ensem_logits_Last",
    use_fp16=True,
    use_openvino=True,
    # opset_version=14
    # base_path="test_onnx"
)

STFT kernels created, time used = 0.1334 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


STFT kernels created, time used = 0.0823 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


STFT kernels created, time used = 0.1184 seconds


...

STFT kernels created, time used = 0.1185 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.


STFT kernels created, time used = 0.1207 seconds


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/nnAudio/features/stft.py:283: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.num_samples < self.pad_amount:
/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/torch/onnx/_internal/jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_

Converting ONNX to OpenVINO
[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression by removing argument "compress_to_fp16" or set it to false "compress_to_fp16=False".
Find more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html
[ SUCCESS ] XML file: ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/onnx_ensem_logits_Last_openvino_fp16/model_simpl.xml
[ SUCCESS ] BIN file: ../logdirs/tf_efficientnetv2_b1_in1k_Exp_noamp_64bs_5sec_PrevCompXCScoredDataNoSecLab_mixupP05_RandomFiltering_balancedSampler_Radamlr1e3_CosBatchLR1e6_Epoch30_FocalLoss_5Folds_NoDuplsV1/onnx_ensem_logits_Last_openvino_fp16/model_simpl.bin


In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
# !rm ../logdirs/convnext_small_fb_in22k_ft_in1k_384__convnextv2_tiny_fcmae_ft_in22k_in1k_384__eca_nfnet_l0_noval_v27_075Clipwise025TimeMax_GausMean -rf

In [ ]:
!ls -lt ../logdirs/ | head

# Check

In [ ]:
os.listdir(f"../logdirs/{EXP_NAME}/")

In [ ]:
core = ov.Core()
model = core.read_model(model=f"../logdirs/{EXP_NAME}/onnx_ensem_openvino_fp16/model_simpl.xml")
compiled_model = core.compile_model(model=model, device_name="CPU")

In [ ]:
sample = np.random.randn(5, 32000*5).astype(np.float16)

In [ ]:
compiled_model.output(0)

In [ ]:
%%time
output = compiled_model([sample])[compiled_model.output(0)]

In [ ]:
output.shape

In [ ]:
output.dtype

In [ ]:
sample = sample.astype(np.float16)

In [ ]:
onnx_model = ort.InferenceSession(
    f"../logdirs/{EXP_NAME}/onnx_ensem_fp16/model_simpl.onnx"
)

In [ ]:
wave = torch.from_numpy(np.random.randn(32000*5)).half()

In [ ]:
wave.numpy()

In [ ]:
%%time
out = onnx_model.run(
    None,
    {"input": sample}
)

In [ ]:
out[0].shape

In [ ]:
if CONFIG["run_test"]:
    inference_class_onnx = BirdsInference(
        device="cpu",
        verbose_tqdm=True,
        use_sigmoid=CONFIG["use_sigmoid"],
        model_output_key=CONFIG["model_output_key"],
    )
    onnx_model = ort.InferenceSession(
        f"../logdirs/{CONFIG['exp_name']}/onnx_ensem_2first_folds/model_simpl.onnx"
    )
    test_preds_onnx, test_preds_long_onnx, test_dfidx_onnx, test_end_onnx = inference_class_onnx.predict_test_loader(
        nn_models=onnx_model,
        data_loader=loader_test,
        is_onnx_model=True
    )
    test_pred_df_onnx = compose_submission_dataframe(
        probs=test_preds_onnx,
        dfidxs=test_dfidx_onnx,
        end_seconds=test_end_onnx,
        filenames=loader_test.dataset.df[loader_test.dataset.name_col].copy(),
        bird2id=bird2id
    )